[![Open In Colab](https://colab.research.google.com/assets/colab-badge.svg)](http://colab.research.google.com/github/sk-classroom/asc-bert/blob/main/assignments/assignment_01.ipynb)

We will learn how to generate word embeddings using BERT. BERT produces contextualized word embeddings, where the embeddings are computed based on the context of the word. Thus, a single word can have different embeddings based on its context. 

# Preparation

In [6]:
# If you haven't installed the required packages, please install them using pip
!pip install transformers plotly

In [8]:
import pandas as pd
import numpy as np
import transformers
import torch
import matplotlib.pyplot as plt
from tqdm import tqdm
from sklearn.decomposition import PCA
import plotly.express as px

# Data 

We will use [CoarseWSD-20](https://github.com/danlou/bert-disambiguation/tree/master/data/CoarseWSD-20). The dataset contains sentences with polysemous words and their sense labels. We will see how to use BERT to disambiguate the word senses.



In [10]:
def load_data(focal_word, is_train, n_samples=100):
    data_type = "train" if is_train else "test"
    data_file = f"https://raw.githubusercontent.com/danlou/bert-disambiguation/master/data/CoarseWSD-20/{focal_word}/{data_type}.data.txt"
    label_file = f"https://raw.githubusercontent.com/danlou/bert-disambiguation/master/data/CoarseWSD-20/{focal_word}/{data_type}.gold.txt"

    data_table = pd.read_csv(
        data_file,
        sep="\t",
        header=None,
        dtype={"word_pos": int, "sentence": str},
        names=["word_pos", "sentence"],
    )
    label_table = pd.read_csv(
        label_file,
        sep="\t",
        header=None,
        dtype={"label": int},
        names=["label"],
    )
    combined_table = pd.concat([data_table, label_table], axis=1)
    return combined_table.sample(n_samples)


focal_word = "apple"

train_data = load_data(focal_word, is_train=True)

train_data.head(10)

,word_pos,sentence,label
617,12,system g is a cluster supercomputer at virgini...,0
1324,22,"in july 2011 , a pedobear iphone application ,...",0
225,8,touch pets : dogs was a game for apple 's ipho...,0
1583,6,"he engaged in farming , especially apple growi...",1
684,15,software packages are downloaded directly to a...,0
1088,27,glenora and adolphustown both remain heavily r...,1
1360,28,therefore pectinase enzymes are commonly used ...,1
349,6,the server is now compatible with apple 's air...,0
2144,32,"for this reason , almost no workstations are b...",0
792,7,weld was noted for numerous and prolific apple...,1


Please refer to the [README](https://github.com/danlou/bert-disambiguation/blob/master/data/CoarseWSD-20/README.txt) for the data. 

## Define BERT model

We will use `transformers` library developed by Hugging Face to define the BERT model. To use the model, we will need:  
1. BERT tokenizer that converts the text into tokens. 
2. BERT model that computes the embeddings of the tokens. 

We will use the `bert-base-uncased` model and tokenizer. Let's define the model and tokenizer. 



In [ ]:
# TODO: Define the model and tokenizer
model = ...
tokenizer = ...

model.eval()

With BERT, we need to prepare text in ways that BERT can understand. 
Specifically, we prepend it with ```[CLS]``` and append ```[SEP]```. We will then convert the text to a tensor of token ids, which is ready to be fed into the model. 



In [ ]:
def prepare_text(text):
    text = "[CLS] " + text + " [SEP]"
    tokenized_text = tokenizer.tokenize(text)
    indexed_tokens = tokenizer.convert_tokens_to_ids(tokenized_text)

    segments_ids = torch.ones((1, len(indexed_tokens)), dtype=torch.long)
    tokens_tensor = torch.tensor([indexed_tokens])
    segments_tensor = segments_ids.clone()
    return tokenized_text, tokens_tensor, segments_tensor

> What is segment tensor?
BERT models are designed to process sentence pairs, differentiated by 0s and 1s to indicate the first and second sentence respectively. In the case of single-sentence inputs, we assign a vector of 1s to each token, indicating they all belong to the first sentence.

Let's get the BERT embeddings for the sentence "Bank is located in the city of London". 

First, let's prepare the text for BERT. 

In [ ]:
text = "Bank is located in the city of London"
tokenized_text, tokens_tensor, segments_tensor = prepare_text(text)

Then, let's get the BERT embeddings for each token. 

In [ ]:
outputs = model(tokens_tensor, segments_tensor)

The output includes `loss`, `logits`, and `hidden_states`. We will use `hidden_states`, which contains the embeddings of the tokens. 



In [ ]:
hidden_states = outputs.hidden_states

print("how many layers? ", len(hidden_states))
print("Shape? ", hidden_states[0].shape)

The hidden states are a list of 13 tensors, where each tensor is of shape (batch_size, sequence_length, hidden_size). The first tensor is the input embeddings, and the subsequent tensors are the hidden states of the BERT layers. 

So, we have 13 choice of hidden states. Deep layers close to the output capture the context of the word from the previous layers.

Here we will take the average over the last four hidden states for each token. 

In [ ]:
# TODO: Compute the embedding of the token
emb = ...
emb.shape

emb is of shape (sequence_length, hidden_size). Let us summarize the embeddings of the tokens into a function. 

In [ ]:
def get_bert_embeddings(text):
    tokenized_text, tokens_tensor, segments_tensor = prepare_text(text)
    outputs = model(tokens_tensor, segments_tensor)
    hidden_states = outputs.hidden_states
    emb = ...
    return emb, tokenized_text

# Embedding
Let's embed the text and get the embedding of the focal word 

In [ ]:
labels = []  # label
emb = []  # embedding
sentences = []  # sentence

# TODO: Go through the data and get the embedding of the focal word.

# Results 

Let's plot the embeddings of the focal word. 

In [ ]:
def plot_result(emb, labels, sentences):
    xy = PCA(n_components=2).fit_transform(emb)

    fig = px.scatter(
        x=xy[:, 0],
        y=xy[:, 1],
        color=labels,
        hover_data=[sentences],
        title="PCA of Word Embeddings",
    )
    fig.update_layout(width=700, height=500)
    fig.update_traces(
        marker=dict(size=12, line=dict(width=2, color="DarkSlateGrey")),
        selector=dict(mode="markers"),
    )
    fig.show()


plot_result(emb, labels, sentences)

# Assignment

In [ ]:
from sklearn.discriminant_analysis import LinearDiscriminantAnalysis


def save_assignment(emb, labels, assignment_id, data_dir):
    K = len(set(labels))
    xy = LinearDiscriminantAnalysis(n_components=K - 1).fit_transform(emb, labels)
    xy_df = pd.DataFrame(xy)
    xy_df["label"] = labels
    xy_df.to_csv(f"{data_dir}/eval_test_{assignment_id}.csv", index=False)

## Assignment 1
- Run this notebook for any word available in [the dataset](https://github.com/danlou/bert-disambiguation/tree/master/data/CoarseWSD-20) except for "apple". 
- Save the (dimensionality-reduced) embeddings of the test data and their labels by running the following cell.  
- Make sure to place the generated file "eval_test_01.csv" in the "data" folder. 
- Commit the file to your assignment repository. 

In [ ]:
# Your code

focal_word = "..."
test_data = load_data(focal_word, is_train=False)

emb = ...
labels = ...

plot_result(emb, labels, sentences)

In [ ]:
save_assignment(emb, labels, assignment_id="01", data_dir="../data")

## Assignment 2

- Use the same dataset as Assignment 1. 
- Compute the word embedding using the first hidden state of the BERT model 
- Save the embeddings of the test data and their labels by running the following cell.  
- Make sure to place the generated file "eval_test_02.csv" in the "data" folder. 
- Commit the file to your assignment repository. 

In [ ]:
# Your code:

emb = ...
labels = ...

plot_result(emb, labels, sentences)

In [ ]:
save_assignment(emb, labels, assignment_id="02", data_dir="../data")